<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [1]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'

--2019-12-09 20:10:34--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.02s   

2019-12-09 20:10:34 (730 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [2]:
import pandas as pd 
df = pd.read_csv('house-votes-84.data', 
                 header=None,
                 names=['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa'])
print(df.shape)
df.head()

(435, 17)


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [3]:
import numpy as np
import seaborn as sns 
df = df.replace({'?':np.NaN, 'n':0, 'y':1})
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [4]:
dem = df[df['party']=='democrat']
rep = df[df['party']=='republican']

print(dem.head(), rep.head())

      party  handicapped-infants  water-project  ...  crime  duty-free  south-africa
2  democrat                  NaN            1.0  ...    1.0        0.0           0.0
3  democrat                  0.0            1.0  ...    0.0        0.0           1.0
4  democrat                  1.0            1.0  ...    1.0        1.0           1.0
5  democrat                  0.0            1.0  ...    1.0        1.0           1.0
6  democrat                  0.0            1.0  ...    1.0        1.0           1.0

[5 rows x 17 columns]          party  handicapped-infants  ...  duty-free  south-africa
0   republican                  0.0  ...        0.0           1.0
1   republican                  0.0  ...        0.0           NaN
7   republican                  0.0  ...        NaN           1.0
8   republican                  0.0  ...        0.0           1.0
10  republican                  0.0  ...        0.0           0.0

[5 rows x 17 columns]


###Testing water project bill 

#### 2 Sample T-test

1). Null hypothesis: 
there is no difference in 2 samples i.e. no difference in how republicans and democrats voted for the water project bill. 

$\bar{x}_1 = \bar{x}_2$

2). Alternative hypothesis (interesting hypothesis): 
there is a difference -- mean voting data is different: 

$\bar{x}_1 \neq \bar{x}_2$

3). Confidence level is 99% per instructions 

In [10]:
#Load libraries
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel
import scipy.stats
from scipy.stats import t
from scipy.stats import ttest_1samp 

#Run t-test

ttest_ind(rep['water-project'], dem['water-project'], nan_policy='omit')

Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

4). T-statistic: 0.09

5). P-value: 0.9291; is p-value < (1-confidence level)? 




In [14]:
0.9291556823993485 <(1-0.99)

False

Due to our test resulting in a t-statistic of 0.09 and having a p-value of 0.93, we fail to reject the null hypothesis that the voting on a water bill is similar for democrats and republicans. 

In [21]:
if(ttest_ind(rep['water-project'], dem['water-project'], nan_policy='omit')[1] <0.01):
  print("We reject null hypothesis")
else: print ("We fail to reject null hypothesis")

We fail to reject null hypothesis


In [18]:
print( dem['water-project'].mean(), rep['water-project'].mean())

0.502092050209205 0.5067567567567568


## let's refactor it to run fast for all bills

In [0]:
#Melt df first 

new_df=df.melt(id_vars='party')
new_df.head()

In [0]:
#Create a functino which would run t-tests on all of the bills in this df
def run_ttests(df):
  bills = df['variable'].unique()
  new_rep = df[df['party']=='republican']
  new_dem = df[df['party']=='democrat']
  for bill in bills:
    ttest_ind(new_rep.value[new_rep['variable']==bill], new_dem.value[new_dem['variable']==bill], nan_policy='omit')
    tstat = ttest_ind(new_rep.value[new_rep['variable']==bill], new_dem.value[new_dem['variable']==bill], nan_policy='omit')[0]
    p=ttest_ind(new_rep.value[new_rep['variable']==bill], new_dem.value[new_dem['variable']==bill], nan_policy='omit')[1]
    if (p < (1-0.99)): 
      print( bill, ': Due to our test resulting in T-statistic = ', tstat, 'and having P-value = ', p, 'we REJECT null hypothesis')
    else: print(bill, ': Due to our test resulting in T-statistic = ', tstat, 'and having P-value = ', p, 'we FAIL TO REJECT null hypothesis')


In [74]:
run_ttests(new_df)

handicapped-infants : Due to our test resulting in T-statistic =  -9.205264294809222 and having P-value =  1.613440327937243e-18 we REJECT null hypothesis
water-project : Due to our test resulting in T-statistic =  0.08896538137868286 and having P-value =  0.9291556823993485 we FAIL TO REJECT null hypothesis
budget : Due to our test resulting in T-statistic =  -23.21277691701378 and having P-value =  2.0703402795404463e-77 we REJECT null hypothesis
physician-fee-freeze : Due to our test resulting in T-statistic =  49.36708157301406 and having P-value =  1.994262314074344e-177 we REJECT null hypothesis
el-salvador-aid : Due to our test resulting in T-statistic =  21.13669261173219 and having P-value =  5.600520111729011e-68 we REJECT null hypothesis
religious-groups : Due to our test resulting in T-statistic =  9.737575825219457 and having P-value =  2.3936722520597287e-20 we REJECT null hypothesis
anti-satellite-ban : Due to our test resulting in T-statistic =  -12.526187929077842 and 

### Create new function which would also look at whether Dems or Republ voted more on a bill, and whether they pass a t-test: 

In [0]:
#Create a functino which would run t-tests on all of the bills in this df and tests how they voted
def ttest_check0(df):
  bills = df['variable'].unique()
  new_rep = df[df['party']=='republican']
  new_dem = df[df['party']=='democrat']
  for bill in bills:
    ttest_ind(new_rep.value[new_rep['variable']==bill], new_dem.value[new_dem['variable']==bill], nan_policy='omit')
    tstat = np.round(ttest_ind(new_rep.value[new_rep['variable']==bill], new_dem.value[new_dem['variable']==bill], nan_policy='omit')[0], 3)
    p= np.round(ttest_ind(new_rep.value[new_rep['variable']==bill], new_dem.value[new_dem['variable']==bill], nan_policy='omit')[1],3)
    rep_support = np.round((new_rep.value[new_rep['variable']==bill].mean())*100)
    dem_support = np.round(new_dem.value[new_dem['variable']==bill].mean()*100)
    if (p > 0.01): 
      print(bill, ' ----- > Statistically, the votes are similar. tstat, p, rep_support, dem_support:',  tstat, p, rep_support, dem_support)
    elif (rep_support > dem_support): 
      print( bill, '----- >  Republicans supported more. tstat, p, rep_support, dem_support: ',  tstat, p, rep_support, dem_support)
    else: print( bill, '----- >  Democrats supported more. tstat, p, rep_support, dem_support: ',  tstat, p, rep_support, dem_support)





In [110]:
ttest_check0(new_df)

handicapped-infants ----- >  Democrats supported more. tstat, p, rep_support, dem_support:  -9.205 0.0 19.0 60.0
water-project  ----- > Statistically, the votes are similar. tstat, p, rep_support, dem_support: 0.089 0.929 51.0 50.0
budget ----- >  Democrats supported more. tstat, p, rep_support, dem_support:  -23.213 0.0 13.0 89.0
physician-fee-freeze ----- >  Republicans supported more. tstat, p, rep_support, dem_support:  49.367 0.0 99.0 5.0
el-salvador-aid ----- >  Republicans supported more. tstat, p, rep_support, dem_support:  21.137 0.0 95.0 22.0
religious-groups ----- >  Republicans supported more. tstat, p, rep_support, dem_support:  9.738 0.0 90.0 48.0
anti-satellite-ban ----- >  Democrats supported more. tstat, p, rep_support, dem_support:  -12.526 0.0 24.0 77.0
aid-to-contras ----- >  Democrats supported more. tstat, p, rep_support, dem_support:  -18.052 0.0 15.0 83.0
mx-missile ----- >  Democrats supported more. tstat, p, rep_support, dem_support:  -16.438 0.0 12.0 76.0
imm

In [0]:
#Create a functino which would run t-tests on all of the bills in this df and tests how they voted
def ttest_check(df):
  result =[]
  bills = df['variable'].unique()
  new_rep = df[df['party']=='republican']
  new_dem = df[df['party']=='democrat']
  for bill in bills:
    ttest_ind(new_rep.value[new_rep['variable']==bill], new_dem.value[new_dem['variable']==bill], nan_policy='omit')
    tstat = np.round(ttest_ind(new_rep.value[new_rep['variable']==bill], new_dem.value[new_dem['variable']==bill], nan_policy='omit')[0], 3)
    p= ttest_ind(new_rep.value[new_rep['variable']==bill], new_dem.value[new_dem['variable']==bill], nan_policy='omit')[1]
    rep_support = np.round((new_rep.value[new_rep['variable']==bill].mean())*100)
    dem_support = np.round(new_dem.value[new_dem['variable']==bill].mean()*100)
    if (p > 0.01): 
      #print(bill, ' ----- > Statistically, the votes are similar. tstat, p, rep_support, dem_support:',  tstat, p, rep_support, dem_support)
      result.append([bill, tstat, p, rep_support, dem_support, 'equal'])
    elif (rep_support > dem_support): 
      #print( bill, '----- >  Republicans supported more. tstat, p, rep_support, dem_support: ',  tstat, p, rep_support, dem_support)
      result.append([bill, tstat, p, rep_support, dem_support, 'rep_more'])
    else: 
      #print( bill, '----- >  Democrats supported more. tstat, p, rep_support, dem_support: ',  tstat, p, rep_support, dem_support)
      result.append([bill, tstat, p, rep_support, dem_support, 'dem_more'])
  return result

In [179]:
ttest_check(new_df)

[['handicapped-infants',
  -9.205,
  1.613440327937243e-18,
  19.0,
  60.0,
  'dem_more'],
 ['water-project', 0.089, 0.9291556823993485, 51.0, 50.0, 'equal'],
 ['budget', -23.213, 2.0703402795404463e-77, 13.0, 89.0, 'dem_more'],
 ['physician-fee-freeze',
  49.367,
  1.994262314074344e-177,
  99.0,
  5.0,
  'rep_more'],
 ['el-salvador-aid', 21.137, 5.600520111729011e-68, 95.0, 22.0, 'rep_more'],
 ['religious-groups', 9.738, 2.3936722520597287e-20, 90.0, 48.0, 'rep_more'],
 ['anti-satellite-ban',
  -12.526,
  8.521033017443867e-31,
  24.0,
  77.0,
  'dem_more'],
 ['aid-to-contras', -18.052, 2.82471841372357e-54, 15.0, 83.0, 'dem_more'],
 ['mx-missile', -16.438, 5.03079265310811e-47, 12.0, 76.0, 'dem_more'],
 ['immigration', 1.736, 0.08330248490425066, 56.0, 47.0, 'equal'],
 ['synfuels', -8.294, 1.5759322301054064e-15, 13.0, 51.0, 'dem_more'],
 ['education', 20.501, 1.8834203990450192e-64, 87.0, 14.0, 'rep_more'],
 ['right-to-sue', 13.511, 1.2278581709672758e-34, 86.0, 29.0, 'rep_more'],


In [172]:
data=pd.DataFrame(ttest_check(new_df), columns=['bill', 'tstat', 'p', 'rep_support', 'dem_support', 'result'])
data

,bill,tstat,p,rep_support,dem_support,result
0,handicapped-infants,-9.205,1.613440e-18,19.0,60.0,dem_more
1,water-project,0.089,9.291557e-01,51.0,50.0,equal
2,budget,-23.213,2.070340e-77,13.0,89.0,dem_more
3,physician-fee-freeze,49.367,1.994262e-177,99.0,5.0,rep_more
4,el-salvador-aid,21.137,5.600520e-68,95.0,22.0,rep_more
5,religious-groups,9.738,2.393672e-20,90.0,48.0,rep_more
6,anti-satellite-ban,-12.526,8.521033e-31,24.0,77.0,dem_more
7,aid-to-contras,-18.052,2.824718e-54,15.0,83.0,dem_more
8,mx-missile,-16.438,5.030793e-47,12.0,76.0,dem_more
9,immigration,1.736,8.330248e-02,56.0,47.0,rep_more


In [174]:
import seaborn as sns
import matplotlib.pyplot as plt

data=data.melt(id_vars=['bill', 'result'])
data

# fix, ax = plt.subplots()
# data.plot()


,bill,result,variable,value
0,handicapped-infants,dem_more,tstat,-9.205
1,water-project,equal,tstat,0.089
2,budget,dem_more,tstat,-23.213
3,physician-fee-freeze,rep_more,tstat,49.367
4,el-salvador-aid,rep_more,tstat,21.137
...,...,...,...,...
59,education,rep_more,dem_support,14.000
60,right-to-sue,rep_more,dem_support,29.000
61,crime,rep_more,dem_support,35.000
62,duty-free,dem_more,dem_support,64.000
